In [1]:
import sys
from pathlib import Path
from os import getcwd

sys.path.append(str(Path(getcwd()).parent))
print(sys.path)

['/home/vinnylg/projects/covid19datascience/notebooks', '/home/vinnylg/anaconda3/lib/python38.zip', '/home/vinnylg/anaconda3/lib/python3.8', '/home/vinnylg/anaconda3/lib/python3.8/lib-dynload', '', '/home/vinnylg/.local/lib/python3.8/site-packages', '/home/vinnylg/anaconda3/lib/python3.8/site-packages', '/home/vinnylg/.local/lib/python3.8/site-packages/IPython/extensions', '/home/vinnylg/.ipython', '/home/vinnylg/projects/covid19datascience']


In [2]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit

In [3]:
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols

default_input: /home/vinnylg/projects/covid19datascience/input
default_output: /home/vinnylg/projects/covid19datascience/output


In [4]:
read_cc = True and False
read_tb = True and False

In [5]:
cc = CasosConfirmados()
if read_cc:
    cc.read('/home/vinnylg/projects/covid19datascience/input/Casos confirmados PR.xlsx')
    cc.save()
else:
    cc.load()

loading Casos Confirmados time elepsed:00:00:01


In [6]:
casosc = cc.get_casos()
del casosc['identificacao']
print(f"casosc len: {len(casosc)}\n")
print(casosc.columns.values)

casosc len: 989417

['ordem' 'ibge7' 'nome' 'sexo' 'idade' 'mun_resid' 'mun_atend' 'rs'
 'laboratorio' 'dt_diag' 'comunicacao' 'is' 'obito' 'data_obito' 'excluir'
 'hash' 'hash_less' 'hash_more' 'hash_atend' 'hash_less_atend'
 'hash_more_atend' 'hash_diag' 'hash_obito']


In [7]:
tb_pacientes = TbPacientes()
if read_tb:
    tb_pacientes.update()

casost = tb_pacientes.get()
print(f"casost len: {len(casost)}\n")
print(casost.columns.values)

casost len: 989417

['identificacao' 'ibge_res_pr' 'ibge_atend_pr' 'nome' 'sexo' 'idade'
 'mun_resid' 'mun_atend' 'laboratorio' 'dt_diag' 'dt_notificacao'
 'dt_inicio_sintomas' '_dia_isolamento' 'dt_viagem_retorno' 'local_viagem'
 'exposicao' 'data_de_internamento' 'local' 'clinico_uti' 'dt_alta'
 'obito' 'dt_obito' 'muni_res_pr' 'hist_viagem' 'casos_novos' 'telefone'
 'dt_atualizacao' 'fonte' 'status' 'eas' 'privado_sus' 'uti'
 'dt_internamento' 'evolucao' 'dt_evolucao' 'temp_permanencia'
 'cod_laboratorio' 'idade_original' 'dt_com_obito' 'dt_com_recuperado'
 'idade_mais' 'idade_menos' 'hash' 'hash_less' 'hash_more' 'hash_atend'
 'hash_less_atend' 'hash_more_atend' 'hash_diag' 'hash_obito']


In [8]:
casost.loc[~(
    casost['hash'].isin(casosc['hash']) |
    casost['hash'].isin(casosc['hash_less']) |
    casost['hash'].isin(casosc['hash_more']) |
    casost['hash_atend'].isin(casosc['hash_atend']) |
    casost['hash_less_atend'].isin(casosc['hash_atend']) |
    casost['hash_more_atend'].isin(casosc['hash_atend']) |
    casost['hash_diag'].isin(casosc['hash_diag'])
)]

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito


In [9]:
tb_cols = ['identificacao','ibge_atend_pr','dt_com_obito','status','dt_com_recuperado','hash_less_atend','hash_more_atend']

casosct = pd.merge(casosc.loc[(casosc['obito']=="SIM")&(casosc['hash'].isin(casost['hash']))],casost[['hash']+tb_cols], how='left', on='hash')
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

23921
965496


In [10]:
casosct =  casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash'])],casost[['hash']+ tb_cols], how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

988359
1674


In [11]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_less'])],casost[['hash_less']+ tb_cols].rename(columns={'hash_less':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

990034
1


In [12]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_more'])],casost[['hash_more']+ tb_cols].rename(columns={'hash_more':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

990034
1


In [13]:
casosct = casosct.append(pd.merge(casosc,casost[['hash_diag']+ tb_cols], how='left', on='hash_diag'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))
casosct.loc[casosct['identificacao'].isna()]

990035
0


,ordem,ibge7,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend_x,hash_more_atend_x,hash_diag,hash_obito,identificacao,ibge_atend_pr,dt_com_obito,status,dt_com_recuperado,hash_less_atend_y,hash_more_atend_y


In [14]:
casosct['identificacao'] = casosct['identificacao'].astype(int)
casost.loc[~casost['identificacao'].isin(casosct['identificacao'])]

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito


In [15]:
id_duplicado = casosct.loc[casosct.duplicated('identificacao',keep=False)].sort_values('hash_diag')
ordem_duplicado = casosct.loc[casosct.duplicated('ordem',keep=False)].sort_values('hash_diag')
print(len(id_duplicado),len(ordem_duplicado))
all(id_duplicado['hash_diag']==ordem_duplicado['hash_diag'])

741 741


True

In [16]:
# casosct.loc[casosct.duplicated('identificacao',keep='last')]
casosct.loc[casosct['comunicacao']==pd.to_datetime('2020-03-11'),'comunicacao'] = pd.to_datetime('2020-03-12')

In [17]:
print(len(casosct))
casosct = casosct.drop_duplicates('identificacao',keep='last')
print('casos:',len(casosct))
print('obitos:',len(casosct.loc[casosct['obito']=='SIM']))

990035
casos: 989417
obitos: 23925


In [18]:
# try:
#     casosct = casosct_bkp
# except:
#     casosct_bkp = casosct
    
casosct = casosct.sort_values(['comunicacao','nome','idade']).copy()

casosct['evolucao'] = 3
casosct.loc[casosct['obito']=='SIM','evolucao'] = 2
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'evolucao'] = 1

casosct.loc[casosct['obito']=='SIM','data_evolucao'] = casosct.loc[casosct['obito'].notna(),'data_obito'] 
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

casosct.loc[casosct['obito']=='SIM','data_com_evolucao'] = casosct.loc[casosct['obito'].notna(),'dt_com_obito'] 
casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_obito']

casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_com_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

municipios = Municipios()
casosct = pd.merge(casosct,municipios[['ibge7','regional','macro','uf','municipio']].rename(columns={'municipio':'municipio_pr'}),how='left',on='ibge7')

casosct.loc[casosct['rs']==99, 'macro'] = 'Fora'
casosct.loc[casosct['rs']==99, 'regional'] = 'Fora'
casosct.loc[casosct['rs']==99, 'uf'] = 'Fora'
casosct.loc[casosct['rs']==99, 'municipio_pr'] = 'Fora'

macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
casosct['macro'] = casosct['macro'].apply(lambda x: macros_labels_idx[x])

# casosct

In [19]:
exclusoes = pd.read_excel('exclusoes.xlsx')
exclusao_casos = casosct.loc[casosct['ordem'].isin(exclusoes['ordem'])].copy()

casosct.loc[exclusao_casos.index,'excluir'] = 'SIM'

casosct.loc[casosct['excluir']=='SIM'].sort_values('nome')

FileNotFoundError: [Errno 2] No such file or directory: 'exclusoes.xlsx'

In [ ]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

total = casosct.groupby(['evolucao','uf'])[['identificacao']].count().unstack().droplevel(0,1)[['PR','Fora']]
total['GERAL'] = total.sum(1)
total = total.append(pd.Series(total.sum(0),name=0))
total.index = [2,1,3,0]
total = total.sort_index()
total = total.rename(index={0:'CASOS',2:'RECUPERADOS',1:'OBITOS',3:'ATIVOS'})
total.index.name = None
total.columns.name = None

total.astype(float)

In [ ]:
total_macros = casosct.groupby(['macro','evolucao'])[['identificacao']].count().unstack()
total_macros.columns = total_macros.columns.droplevel(0)

total_macros['CASOS'] = total_macros.sum(1)
total_macros = total_macros.fillna(0)
total_macros = total_macros.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_macros = total_macros[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_macros.index = [ macros_idx_labels[x] if isinstance(x,int) else x for x in total_macros.index]

## comentar 3 linhas abaixo para remover totais
# total_macros = total_macros.append(pd.Series(total_macros.sum(0),name=('TOTAL GERAL')))
# pr = total_macros.loc[('TOTAL GERAL')].values - total_macros.loc['Fora'].values
# total_macros = total_macros.append(pd.Series(pr, index=total_macros.columns, name=(('TOTAL PR'))))

total_macros.astype(float)

In [ ]:
total_regionais = casosct.groupby(['rs','regional','evolucao'])[['identificacao']].count().unstack()
total_regionais.columns = total_regionais.columns.droplevel(0)

total_regionais['CASOS'] = total_regionais.sum(1)
total_regionais = total_regionais.fillna(0)
total_regionais = total_regionais.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_regionais = total_regionais[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

# # comentar 3 linhas abaixo para remover totais
# total_regionais = total_regionais.append(pd.Series(total_regionais.sum(0),name=('TOTAL','GERAL')))
# pr = (total_regionais.loc[('TOTAL','GERAL')].values - total_regionais.loc[99].values)[0]
# total_regionais = total_regionais.append(pd.Series(pr, index=total_regionais.columns, name=(('TOTAL','PR'))))

total_regionais.astype(float)

In [ ]:
total_municipios = casosct.groupby(['macro','rs','regional','ibge7','municipio_pr','evolucao'])[['identificacao']].count().unstack()

total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_municipios = total_municipios[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_municipios.index = total_municipios.index.set_levels([ macros_idx_labels[x] if isinstance(x,int) else x for x in total_municipios.index.levels[0]], level=0)

# #comentar 3 linhas abaixo para remover totais
# total_municipios = total_municipios.append(pd.Series(total_municipios.sum(0),name=('','TOTAL','GERAL','','')))
# pr = (total_municipios.loc[('','TOTAL','GERAL','','')].values - total_municipios.loc['Fora'].values)[0]
# total_municipios = total_municipios.append(pd.Series(pr, index=total_municipios.columns, name=(('','TOTAL','PR','',''))))

total_municipios.astype(float)

In [ ]:
# casosct['macro'] = casosct['macro'].apply(lambda x: macros_idx_labels[x])

In [ ]:
# casosct = casosct[['ordem','identificacao','uf','macro','ibge7','ibge_atend_pr','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','excluir']]
# obitosct = casosct.loc[casosct['evolucao']==2].sort_values(['data_com_evolucao','nome','idade'])
# obitosct = obitosct[['ordem','identificacao','uf','macro','ibge7','nome','sexo','idade','mun_resid','rs','data_evolucao','data_com_evolucao','excluir']]

In [ ]:
casos = casosct[['identificacao','ordem', 'ibge7','macro','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','excluir','evolucao','data_com_evolucao']]
obitos = casos.loc[casos['evolucao']==2].sort_values(['data_com_evolucao','nome','idade'])
obitos = obitos[['ordem','ibge7','macro','nome','sexo','idade','mun_resid','rs','data_obito','excluir']]

In [ ]:
writer = pd.ExcelWriter(join("output","novos_casos_confirmados.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

casosct.to_excel(writer,f'casos', index=False)
fit_cols(writer,casosct,f'casos')

obitos.to_excel(writer,f'obitos',index=False)
fit_cols(writer,obitos,f'obitos')

writer.save()

In [ ]:
writer = pd.ExcelWriter(join("output","casos_confirmados.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx,macro in macros_idx_labels.items():
    casos_macro = casos.loc[casos['macro']==idx].copy()
    casos_macro['macro'] = macro

    casos_macro.to_excel(writer,macro,index=False)
    fit_cols(writer,casos_macro,macro)
    
writer.save()

In [ ]:
writer = pd.ExcelWriter(join("output","obitos_confirmados.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx,macro in macros_idx_labels.items():
    obitos_macro = obitos.loc[obitos['macro']==idx].copy()
    obitos_macro['macro'] = macro
    
    obitos_macro.to_excel(writer,macro,index=False)
    fit_cols(writer,obitos_macro,macro)
    
writer.save()